In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
import pandas as pd

In [5]:
# Set up WebDriver (assuming you're using Chrome)
options = webdriver.ChromeOptions()
options.add_experimental_option('prefs', {
    "download.default_directory": "C:/Users/luisf/Downloads/propuestas_politicas",  # Change path accordingly
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
})

driver = webdriver.Chrome(options=options)

driver.get('https://candidaturas2021.ine.mx/')

In [12]:
dropdown = driver.find_element(By.CSS_SELECTOR, 'div.filtro:nth-child(4) > div:nth-child(1) > div:nth-child(1)')

# Click the dropdown to reveal the options
dropdown.click()



In [13]:
dropdown_container = driver.find_element(By.CLASS_NAME, 'rc-virtual-list-holder')

dropdown_container.click()

# Give it a moment to load options
time.sleep(2)

# Locate all the dropdown options (div elements that represent the options)
options = driver.find_elements(By.CLASS_NAME, 'col-xs-12 espaciador-bottom-xs filtro')

# Iterate over the options and select each one
for option in options:
    print(f"Selecting option: {option.text}")

# Close the browser after the task is done
driver.quit()